In [3]:
# RNN 

In [4]:
# 데이터 - 토큰화 & 정수 인코딩 - 시퀀스 패딩
# baseline 일반 신경망 Dense
# SimpleRNN  기울기 소실문제
# Bidirecitional LSTM :  양방향 학습

In [5]:
# Tokenization  텍스트를 수치로 변환
# Word Embedding  단어의 의미를 벡터로 표현
# Sequence Padding  시퀀스 길이 맞추기
# SimpleRNN  순환 신경망
# LSTM      RNN의 경사소실문제 해결
# Bidirectional LSTM  양방향 컨텐츠 학습
# Binary Classification  이진 분류

In [6]:
# 토큰화 : 숫자로 변환하는 과정
# 토크나이저 3단계
    # 1. fit_on_texts()  가장 빈도가 높은 단어의 인덱스를 구축해서 딕셔너리
    # 2. texts_to_sequences()  텍스트를 정수 인덱스 시퀀스로 변환
    # 3. pad_sequences()  시퀀스 길이 맞추기 (패딩)

In [7]:
# 딥러닝에서 워드 임배딩 레이어 : 각 단어를 고정된 크기의 실수 벡터


In [8]:
sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

In [9]:
# 파이토치 버전
# 토큰화
from collections import Counter
import numpy as np
import torch


In [10]:
# 1. 단어 분할 및 빈도 계산
all_words = []
for i in [review.split() for review in sample_reviews]:
    all_words.extend(i)

In [11]:
# 단어 빈도
word_freq = Counter(all_words)

In [12]:
# 2. Tokkenizer 구현
class  SimpleTokenizer:
    def __init__(self, num_words=10, oov_token="UNK"):
        self.num_words = num_words
        self.oov_token = oov_token
        self.word_index = {}
        self.index_word = {}
    
    def fit_on_texts(self, texts):
        all_words = []
        for i in [review.split() for review in texts]:
            all_words.extend(i)
        word_freq = Counter(all_words)
        # 빈도 높은 순서로 인덱스 부여
        # oov 토큰을 1로 설정
        self.word_index[self.oov_token] = 1
        self.index_word[1] = self.oov_token
        idx = 2
        for word, freq in word_freq.most_common(self.num_words - 1):
            self.word_index[word] = idx
            self.index_word[idx] = word
            idx += 1
    def texts_to_sequences(self, texts):
        '''텍스트를 정수 시퀀스로 변환'''
        sequences = []
        for text in texts:
            seq = []
            for word in text.split():
                # 단어가 vocalulary에 있으면 인덱스를 사용 없으면 oov
                word_index = self.word_index.get(word, 1)
                seq.append(word_index)
            sequences.append(seq)
        return sequences
# Tokenizer 생성 및 학습
tokenizer = SimpleTokenizer(num_words=10, oov_token="UNK")
tokenizer.fit_on_texts(sample_reviews)
print("단어 인덱스:", tokenizer.word_index)
# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sample_reviews)
print("정수 시퀀스:", sequences)

단어 인덱스: {'UNK': 1, 'movie': 2, 'great': 3, 'wonderful': 4, 'this': 5, 'is': 6, 'and': 7, 'bad': 8, 'with': 9, 'poor': 10}
정수 시퀀스: [[5, 2, 6, 3, 7, 4], [8, 2, 9, 10, 1], [3, 2, 1, 4]]


In [14]:
# 패딩 구현 - 문자열의 길이를 동일하게 맞춘다
def pad_sequence_manual(squence, max_len=10, padding='pre', value=0):
    '''패딩구현'''
    padded = []
    for seq in squence:
        if len(seq) >= max_len:
            if padding == 'pre':
                padded_seq = seq[-max_len:]
            else:
                padded_seq = seq[:max_len]
        else:
            pad_length = max_len - len(seq)
            if padding == 'pre':
                padded_seq = [value] * pad_length + seq
            else:
                padded_seq = seq + [value] * pad_length
        padded.append(padded_seq)
    return np.array(padded)
# 패딩 적용
padded_sequences = pad_sequence_manual(sequences, max_len=10, padding='pre', value=0)
print("패딩된 시퀀스:\n", padded_sequences)

패딩된 시퀀스:
 [[ 0  0  0  0  5  2  6  3  7  4]
 [ 0  0  0  0  0  8  2  9 10  1]
 [ 0  0  0  0  0  0  3  2  1  4]]


In [17]:
# pyTorch 텐서로 변환
sequence_tensor = torch.LongTensor(padded_sequences)
sequence_tensor

tensor([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
        [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
        [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [27]:
# 2. 워드 임배딩 - 시각화
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.nn as nn
print(f'패딩된 시퀀스 형태 : {padded_sequences.shape}')
print(f'첫번째 : {sequence_tensor[0]}')
# pythorch emvedding 레이어 생성
# num_embeddings 어휘의 크기
# embedding_dim 각 단어를 몇차원 벡터로 표현할 것인지
emvbedding_layer = nn.Embedding(num_embeddings=1000, embedding_dim=8, padding_idx=0)
embedded = emvbedding_layer(sequence_tensor)
print(f'입력형태 : {sequence_tensor.shape}')
print(f'출력형태 : {embedded.shape}')


패딩된 시퀀스 형태 : (3, 10)
첫번째 : tensor([0, 0, 0, 0, 5, 2, 6, 3, 7, 4])
입력형태 : torch.Size([3, 10])
출력형태 : torch.Size([3, 10, 8])


In [33]:
# 임베딩 벡터 상세 분석
# 데이터 3개
for word_idx in range(3):
    embedding_vec = embedded[0, word_idx].detach().numpy()
    word_id = sequence_tensor[0, word_idx].item()
    print(f'단어 ID: {word_id}, 임베딩 벡터: {embedding_vec}')

단어 ID: 0, 임베딩 벡터: [0. 0. 0. 0. 0. 0. 0. 0.]
단어 ID: 0, 임베딩 벡터: [0. 0. 0. 0. 0. 0. 0. 0.]
단어 ID: 0, 임베딩 벡터: [0. 0. 0. 0. 0. 0. 0. 0.]


In [35]:
# 임베딩 행렬
emvbedding_matrix = emvbedding_layer.weight.detach().numpy()
print(f'입베딩 행렬 형태 : {emvbedding_matrix.shape}')
print(f'패딩 id = 0의 임베딩 벡터 : {emvbedding_matrix[0]}')
print(f'단어 id = 5인 임베딩 벡터 : {emvbedding_matrix[5]}')

입베딩 행렬 형태 : (1000, 8)
패딩 id = 0의 임베딩 벡터 : [0. 0. 0. 0. 0. 0. 0. 0.]
단어 id = 5인 임베딩 벡터 : [ 0.49397928 -0.5430416   0.9405944  -1.8554078   0.45127866  0.9239473
 -0.33081326  1.3918082 ]


In [36]:
# ---

In [39]:
# RNN 적용
class RnnModule(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RnnModule, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, _ = self.rnn(embedded)
        last_hidden_state = rnn_out[:, -1, :]
        out = self.fc(last_hidden_state)
        out = self.sigmoid(out)
        return out
